In [1]:
import pandas as pd
import numpy as np
import random
import operator
import math

In [3]:
df_full = pd.read_csv("SPECTF_New.csv")
columns = list(df_full.columns)
features = columns[:len(columns)-1]
class_labels = list(df_full[columns[-1]])
df = df_full[features]

# Number of Attributes
num_attr = len(df.columns) - 1

# Number of Clusters
k = 2

# Maximum number of iterations
MAX_ITER = 100

# Number of data points
n = len(df)

# Fuzzy parameter
m = 2.00

In [4]:
def accuracy(cluster_labels, class_labels):
        county = [0,0]
        countn = [0,0]
        tp = [0, 0]
        tn = [0, 0]
        fp = [0, 0]
        fn = [0, 0]
    
        for i in range(len(df)):
            # Yes = 1, No = 0
            if cluster_labels[i] == 1 and class_labels[i] == 'Yes':
                tp[0] = tp[0] + 1
            if cluster_labels[i] == 0 and class_labels[i] == 'No':
                tn[0] = tn[0] + 1
            if cluster_labels[i] == 1 and class_labels[i] == 'No':
                fp[0] = fp[0] + 1
            if cluster_labels[i] == 0 and class_labels[i] == 'Yes':
                fn[0] = fn[0] + 1
    
        for i in range(len(df)):
            # Yes = 0, No = 1
            if cluster_labels[i] == 0 and class_labels[i] == 'Yes':
                tp[1] = tp[1] + 1
            if cluster_labels[i] == 1 and class_labels[i] == 'No':
                tn[1] = tn[1] + 1
            if cluster_labels[i] == 0 and class_labels[i] == 'No':
                fp[1] = fp[1] + 1
            if cluster_labels[i] == 1 and class_labels[i] == 'Yes':
                fn[1] = fn[1] + 1
    
        a0 = float((tp[0] + tn[0]))/(tp[0] + tn[0] + fn[0] + fp[0])
        a1 = float((tp[1] + tn[1]))/(tp[1] + tn[1] + fn[1] + fp[1])
        p0 = float(tp[0])/(tp[0] + fp[0])
        p1 = float(tp[1])/(tp[1] + fp[1])
        r0 = float(tp[0])/(tp[0] + fn[0])
        r1 = float(tp[1])/(tp[1] + fn[1])
    
        accuracy = [a0*100,a1*100]
        precision = [p0*100,p1*100]
        recall = [r0*100,r1*100]
    
        return accuracy, precision, recall

In [5]:
def initializeMembershipMatrix():
        membership_mat = list()
        for i in range(n):
            random_num_list = [random.random() for i in range(k)]
            summation = sum(random_num_list)
            temp_list = [x/summation for x in random_num_list]
            membership_mat.append(temp_list)
        return membership_mat

In [6]:
def calculateClusterCenter(membership_mat):
        cluster_mem_val = zip(*membership_mat)
        cluster_centers = list()
        for j in range(k):
            x = list(cluster_mem_val[j])
            xraised = [e ** m for e in x]
            denominator = sum(xraised)
            temp_num = list()
            for i in range(n):
                data_point = list(df.iloc[i])
                prod = [xraised[i] * val for val in data_point]
                temp_num.append(prod)
            numerator = map(sum, zip(*temp_num))
            center = [z/denominator for z in numerator]
            cluster_centers.append(center)
        return cluster_centers

In [7]:
def updateMembershipValue(membership_mat, cluster_centers):
        p = float(2/(m-1))
        for i in range(n):
            x = list(df.iloc[i])
            distances = [np.linalg.norm(map(operator.sub, x, cluster_centers[j])) for j in range(k)]
            for j in range(k):
                den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
                membership_mat[i][j] = float(1/den)       
        return membership_mat

In [8]:
def getClusters(membership_mat):
        cluster_labels = list()
        for i in range(n):
            max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
            cluster_labels.append(idx)
        return cluster_labels

In [9]:
def fuzzyCMeansClustering():
        # Membership Matrix
        membership_mat = initializeMembershipMatrix()
        curr = 0
        while curr <= MAX_ITER:
            cluster_centers = calculateClusterCenter(membership_mat)
            membership_mat = updateMembershipValue(membership_mat, cluster_centers)
            cluster_labels = getClusters(membership_mat)
            curr += 1
        print(membership_mat)
        return cluster_labels, cluster_centers
    

In [10]:
labels, centers = fuzzyCMeansClustering()
a,p,r = accuracy(labels, class_labels)

print("Accuracy = " + str(a))
print("Precision = " + str(p))
print("Recall = " + str(r))

[[0.5023453267780242, 0.4976546732219758], [0.49866666624509176, 0.5013333337549082], [0.4997873734791836, 0.5002126265208166], [0.49850805477053517, 0.5014919452294648], [0.5010833153379292, 0.4989166846620708], [0.502949933167043, 0.49705006683295705], [0.5017211218782395, 0.49827887812176047], [0.5022860876022252, 0.49771391239777474], [0.5007554088711271, 0.49924459112887304], [0.5031103048975416, 0.4968896951024583], [0.5016619942034886, 0.4983380057965114], [0.5014540604154089, 0.49854593958459115], [0.49949299121324, 0.50050700878676], [0.5021832013977544, 0.49781679860224554], [0.5024168973962583, 0.4975831026037418], [0.5013282737832052, 0.49867172621679484], [0.50334154612881, 0.49665845387119], [0.5008625869810218, 0.4991374130189781], [0.5005783224956559, 0.4994216775043441], [0.497644027686899, 0.502355972313101], [0.5025194372196076, 0.49748056278039254], [0.5027501861189013, 0.49724981388109873], [0.4978177401257798, 0.5021822598742202], [0.5020466972132183, 0.4979533027